# 第5回講義 宿題

## 課題. Theanoを用いて, MNISTを多層パーセプトロン(MLP)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください

- CNNは使わないでください

次のような内容のコードが**事前**に実行されます

```python
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np
import theano
import theano.tensor as T


mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data.astype('float32'),
                           mnist.target.astype('int32'), random_state=42)

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y,
                                                    test_size=0.2,
                                                    random_state=??)

```

次のセルのhomework関数を完成させて提出してください
- **上記のコード以外で必要なもの**は全て書いてください

In [9]:
def homework(train_X, train_y, test_X):
    # Multi Layer Perceptron
    train_y = np.eye(10)[train_y]
    train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y,
                                                          test_size=0.2,
                                                          random_state=None)
    class Layer:
        # Constructor
        def __init__(self, in_dim, out_dim, function, possibility, dropout = True):
            self.in_dim = in_dim
            self.out_dim = out_dim
            self.function = function
            self.W = theano.shared(np.random.uniform(low=-0.10, high=0.08,
                                               size=(in_dim, out_dim)
                                               ).astype('float32'), name='W')
            self.b = theano.shared(np.random.uniform(low =-0.10,high = 0.08,
                                                     size = out_dim).astype('float32'), name='b')
            self.possibility = possibility
            self.mask =[]
            self.params = [self.W, self.b]
            #self.momentum =　momentum

        # Forward Propagation
        def f_prop(self, x):
            #self.mask = self.get_mask()
            self.z = self.function(T.dot(x, self.W) + self.b)
            return self.z
        
        def get_mask(self):
            a = np.random.rand(self.out_dim) < self.possibility
            return a*np.float32(1.0)
        
    # Stochastic Gradient Descent
    def sgd(params, g_params, eps=np.float32(0.1)):
        updates = OrderedDict()
        ##momentum updates
        for param, g_param in zip(params, g_params):
            updates[param] = param - eps*g_param
            
        #print(updates)
        return updates
    
    layers = [
        Layer(784, 200, T.nnet.sigmoid, 0.80),
        Layer(200, 100, T.nnet.sigmoid, 0.60),
        Layer(100, 10, T.nnet.softmax, 1.00)]

    x = T.fmatrix('x')
    t = T.imatrix('t')

    params = []
    for i, layer in enumerate(layers):
        params += layer.params
        # print (params[0].get_value(),params[1].get_value())
        if i == 0:
            layer.mask = layer.get_mask()
            layer_out = layer.f_prop(x)*layer.mask 
        else:
            layer.mask = layer.get_mask()
            layer_out = layer.f_prop(layer_out)*layer.mask 
    y = layers[-1].z
    
    for i, layer in enumerate(layers):
        if i == 0:
            layer_out = layer.f_prop(x)*layer.possibility
        else:
            layer_out = layer.f_prop(layer_out)*layer.possibility

    result = layers[-1].z
    
    cost = T.mean(T.nnet.categorical_crossentropy(y, t))

    g_params = T.grad(cost=cost, wrt=params)
    
    updates = sgd(params, g_params)

    train = theano.function(inputs=[x, t], outputs=cost, updates=updates,
                            allow_input_downcast=True, name='train')
    valid = theano.function(inputs=[x, t], outputs=[cost, T.argmax(y, axis=1)],
                            allow_input_downcast=True, name='valid')
    test = theano.function(inputs=[x], outputs=T.argmax(result, axis=1), name='test')

    batch_size = 100
    n_batches = train_X.shape[0]//batch_size
    for epoch in range(150):
        train_X, train_y = shuffle(train_X, train_y)
        for i in range(n_batches):
            start = i*batch_size
            end = start + batch_size
            train(train_X[start:end], train_y[start:end])
        valid_cost, pred_y = valid(valid_X, valid_y)
        #if epoch % 10 == 9:
         #   print('EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f' %
        #        (epoch + 1, valid_cost,
        #           f1_score(np.argmax(valid_y, axis=1).astype('int32'),
                     #pred_y, average='macro')))
    pred_y = test(test_X)
    #mistakelist = np.where((pred_y != test_y)==1)[0]
    
  
    
    return pred_y

In [2]:
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np
import theano
import theano.tensor as T


mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data.astype('float32'),
                           mnist.target.astype('int32'), random_state=42)

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y,
                                                    test_size=0.2,
                                                    random_state=None)

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)


In [5]:
homework(train_X,train_y,test_X)#0.70,0.50

EPOCH:: 10, Validation cost: 0.343, Validation F1: 0.901
EPOCH:: 20, Validation cost: 0.236, Validation F1: 0.932
EPOCH:: 30, Validation cost: 0.176, Validation F1: 0.949
EPOCH:: 40, Validation cost: 0.144, Validation F1: 0.958
EPOCH:: 50, Validation cost: 0.124, Validation F1: 0.963
EPOCH:: 60, Validation cost: 0.111, Validation F1: 0.965
EPOCH:: 70, Validation cost: 0.103, Validation F1: 0.968
EPOCH:: 80, Validation cost: 0.095, Validation F1: 0.970
EPOCH:: 90, Validation cost: 0.091, Validation F1: 0.972
EPOCH:: 100, Validation cost: 0.088, Validation F1: 0.972
EPOCH:: 110, Validation cost: 0.086, Validation F1: 0.973
EPOCH:: 120, Validation cost: 0.085, Validation F1: 0.973
EPOCH:: 130, Validation cost: 0.085, Validation F1: 0.974
EPOCH:: 140, Validation cost: 0.084, Validation F1: 0.975
EPOCH:: 150, Validation cost: 0.084, Validation F1: 0.975
EPOCH:: 160, Validation cost: 0.084, Validation F1: 0.975
EPOCH:: 170, Validation cost: 0.084, Validation F1: 0.975
EPOCH:: 180, Validation

[array([7, 1, 6, ..., 7, 0, 9]),
 0.9722142857142857,
 array([7, 1, 6, ..., 7, 0, 9], dtype=int32)]

In [7]:
homework(train_X,train_y,test_X)

EPOCH:: 10, Validation cost: 0.325, Validation F1: 0.904
EPOCH:: 20, Validation cost: 0.218, Validation F1: 0.936
EPOCH:: 30, Validation cost: 0.167, Validation F1: 0.951
EPOCH:: 40, Validation cost: 0.138, Validation F1: 0.958
EPOCH:: 50, Validation cost: 0.120, Validation F1: 0.964
EPOCH:: 60, Validation cost: 0.109, Validation F1: 0.967
EPOCH:: 70, Validation cost: 0.100, Validation F1: 0.969
EPOCH:: 80, Validation cost: 0.098, Validation F1: 0.969
EPOCH:: 90, Validation cost: 0.094, Validation F1: 0.970
EPOCH:: 100, Validation cost: 0.090, Validation F1: 0.972
EPOCH:: 110, Validation cost: 0.089, Validation F1: 0.972
EPOCH:: 120, Validation cost: 0.090, Validation F1: 0.972
EPOCH:: 130, Validation cost: 0.089, Validation F1: 0.974
EPOCH:: 140, Validation cost: 0.090, Validation F1: 0.972
EPOCH:: 150, Validation cost: 0.090, Validation F1: 0.973
EPOCH:: 160, Validation cost: 0.090, Validation F1: 0.973
EPOCH:: 170, Validation cost: 0.091, Validation F1: 0.974
EPOCH:: 180, Validation

[array([7, 1, 6, ..., 7, 0, 9]),
 0.97642857142857142,
 array([7, 1, 6, ..., 7, 0, 9], dtype=int32)]

In [10]:
homework(train_X,train_y,test_X)

array([7, 1, 6, ..., 7, 0, 9])

In [ ]:
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np
import theano
import theano.tensor as T


def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data.astype('float32'),
                               mnist.target.astype('int32'), random_state=42)

    mnist_X = mnist_X / 255.0

    train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y,
                                                        test_size=0.2,
                                                        random_state=42)

    return (train_X, test_X, train_y, test_y)


def check_homework():
    train_X, test_X, train_y, test_y = load_mnist()

    # validate for small dataset
    train_X_mini = train_X[:1000]
    train_y_mini = train_y[:1000]
    test_X_mini = test_X[:1000]
    test_y_mini = test_y[:1000]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    return f1_score(test_y_mini, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()

    print("No Error Occured!")